# Tutorial 1: Core PADE

This tutorial covers the core `pade` package only. No PDK-specific code, no external utilities.
The goal is to learn the API mechanics so you understand what the core gives you — and what it
deliberately leaves for you to build on top.

Simulation examples use NGspice. An equivalent Spectre backend exists, and support for
additional proprietary tools will be added in the future.

**Contents:**
1. [Schematic: RC Lowpass Filter](#1.-Schematic:-RC-Lowpass-Filter)
2. [Transient Simulation](#2.-Transient-Simulation)
3. [AC Simulation](#3.-AC-Simulation)
4. [Layout Fundamentals](#4.-Layout-Fundamentals)
5. [Hierarchy, Transforms, Placement](#5.-Hierarchy,-Transforms,-Placement)
6. [Routing](#6.-Routing)
7. [GDS Export & Import](#7.-GDS-Export-&-Import)
8. [Schematic–Layout Integration](#8.-Schematic–Layout-Integration)
9. [Summary](#9.-Summary)

In [1]:
from pathlib import Path
import numpy as np

from bokeh.plotting import figure, show
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

---
## 1. Schematic: RC Lowpass Filter

A schematic in PADE is a tree of `Cell` objects. Each cell has **terminals** (interface ports),
**nets** (internal wires), **subcells** (child instances), and **parameters**.

The standard library (`pade.stdlib`) provides ideal components common to most circuit simulators: `V`, `R`, `C`, `L`, `I`, `B`.

In [2]:
from pade.core.cell import Cell
from pade.core.testbench import Testbench
from pade.stdlib import V, R, C

### Defining a cell

Subclass `Cell` and define terminals, subcells, and connections in `__init__`.
Assigning a `Cell` to `self.<name>` automatically sets its `instance_name` and registers it as a subcell.

In [3]:
class RCFilter(Cell):
    def __init__(self, r=1e3, c=1e-12, **kwargs):
        super().__init__(**kwargs)
        self.add_terminal(['inp', 'out', 'vss'])

        # Instantiate subcells — attribute name becomes instance_name
        self.R1 = R(r=r)
        self.C1 = C(c=c)

        # Connect subcell terminals to nets (created in self)
        # subcell.connect([terminal_names], [net_names])
        self.R1.connect(['p', 'n'], ['inp', 'out'])
        self.C1.connect(['p', 'n'], ['out', 'vss'])

### Building a testbench

`Testbench` is a `Cell` with `instance_name == cell_name` (marking it as top-level)
and a ground net `'0'` added by default.

In [4]:
class RCFilterTB(Testbench):
    def __init__(self):
        super().__init__()

        self.DUT = RCFilter(r=10e3, c=10e-12)
        self.VIN = V(dc=0, type='pulse', v1=0, v2=1.8, tr=1e-9, tf=1e-9, pw=50e-9, per=100e-9)

        self.VIN.connect(['p', 'n'], ['inp', '0'])
        self.DUT.connect(['inp', 'out', 'vss'], ['inp', 'out', '0'])

tb = RCFilterTB()

### Inspecting the hierarchy

The cell tree is plain Python data structures — dicts of subcells, terminals, nets, parameters.

In [5]:
print('Top-level subcells:', list(tb.subcells.keys()))
print('DUT subcells:      ', list(tb.DUT.subcells.keys()))
print('DUT terminals:     ', list(tb.DUT.terminals.keys()))
print('DUT nets:          ', {k: v.name for k, v in tb.DUT.nets.items()})
print('R1 parameters:     ', {k: v.value for k, v in tb.DUT.R1.parameters.items()})
print('R1 terminal nets:  ', {t.name: t.net.name for t in tb.DUT.R1.get_all_terminals()})
print('Hierarchy name:    ', tb.DUT.R1.get_name_from_top())

Top-level subcells: ['DUT', 'VIN']
DUT subcells:       ['R1', 'C1']
DUT terminals:      ['inp', 'out', 'vss']
DUT nets:           {'inp': 'inp', 'out': 'out', 'vss': 'vss'}
R1 parameters:      {'r': 10000.0}
R1 terminal nets:   {'p': 'inp', 'n': 'out'}
Hierarchy name:     DUT.R1


---
## 2. Transient Simulation

Simulation is controlled by `Statement` objects (analysis, save, include, etc.).
The `SpiceNetlistWriter` generates a SPICE netlist string from the cell hierarchy,
and `NgspiceSimulator` runs it.

In [6]:
from pade.statement import Analysis, Save
from pade.backends.ngspice.netlist_writer import SpiceNetlistWriter
from pade.backends.ngspice.simulator import NgspiceSimulator
from pade.backends.ngspice.results_reader import read_raw

### Generating a netlist

Before running a simulation, inspect the generated netlist.

In [7]:
statements = [
    Analysis('tran', stop=200e-9),
    Save(['inp', 'out']),
]

writer = SpiceNetlistWriter()
print(writer.generate_netlist(tb, statements))

* RCFilterTB - Generated by PADE

.subckt RCFilter inp out vss
R1 inp out 10k
C1 out vss 10p
.ends RCFilter

* Top-level instances
XDUT inp out 0 RCFilter
VIN inp 0 pulse(0 1.8 0 1n 1n 50n 100n)

.tran 200p 200n
.save v(inp) v(out)
.options filetype=ascii
.end


### Running the simulation

In [8]:
WORK_DIR = Path('../work')

sim = NgspiceSimulator(output_dir=WORK_DIR / 'sim')
raw_path = sim.simulate(tb, statements, 'rc_tran', show_output=False)

[INFO] Netlist written to ../work/sim/rc_tran/RCFilterTB.spice
[INFO] Running: ngspice -b ../work/sim/rc_tran/RCFilterTB.spice -r ../work/sim/rc_tran/output.raw


[INFO] NGspice complete in 0.9s


### Reading and plotting results

`read_raw` parses the ASCII raw file into a dict of numpy arrays.

In [9]:
data = read_raw(raw_path)
print('Available signals:', list(data.keys()))

Available signals: ['time', 'v(inp)', 'v(out)']


In [10]:
t = data['time'] * 1e9  # convert to ns

p = figure(title='RC Filter — Transient', width=700, height=300,
           x_axis_label='Time [ns]', y_axis_label='Voltage [V]')
p.line(t, data['v(inp)'], legend_label='inp', line_color='steelblue')
p.line(t, data['v(out)'], legend_label='out', line_color='tomato')
p.legend.location = 'top_right'
show(p)

---
## 3. AC Simulation

The same schematic can be reused with different analysis statements.
For AC analysis, we need a small-signal source. Let's build a new testbench with an AC stimulus.

In [11]:
class RCFilterAC_TB(Testbench):
    def __init__(self):
        super().__init__()

        self.DUT = RCFilter(r=10e3, c=10e-12)
        self.VIN = V(dc=0, ac=1)  # AC stimulus: 1V magnitude

        self.VIN.connect(['p', 'n'], ['inp', '0'])
        self.DUT.connect(['inp', 'out', 'vss'], ['inp', 'out', '0'])

tb_ac = RCFilterAC_TB()

statements_ac = [
    Analysis('ac', start=1e3, stop=10e9, variation='dec', points=20),
    Save(['out']),
]

raw_ac = sim.simulate(tb_ac, statements_ac, 'rc_ac', show_output=False)

[INFO] Netlist written to ../work/sim/rc_ac/RCFilterAC_TB.spice
[INFO] Running: ngspice -b ../work/sim/rc_ac/RCFilterAC_TB.spice -r ../work/sim/rc_ac/output.raw
[INFO] NGspice complete in 0.8s


In [12]:
data_ac = read_raw(raw_ac)
freq = data_ac['frequency']
vout = data_ac['v(out)']

mag_db = 20 * np.log10(np.abs(vout))
phase_deg = np.degrees(np.angle(vout))

p = figure(title='RC Filter — AC Response', width=700, height=300,
           x_axis_label='Frequency [Hz]', y_axis_label='Magnitude [dB]',
           x_axis_type='log')
p.line(freq, mag_db, line_color='steelblue')
show(p)

---
## 4. Layout Fundamentals

Layout in PADE is built from `LayoutCell` objects containing `Shape` objects on `Layer` objects.
All coordinates are in **nanometers** (integers).

A `Ref` is a named pointer to a shape — used as a routing anchor.
A `Pin` is a `Ref` with LVS significance — it maps to a schematic terminal.

In [13]:
from pade.layout.cell import LayoutCell
from pade.layout.shape import Layer, Shape, LayerMap
from pade.layout.ref import Ref, Pin

### Layers and shapes

Define layers with a name and purpose. In a real PDK, these would map to GDS layer numbers.
The `connectivity` flag marks layers that carry electrical signal (used by `check_shorts`).

In [14]:
M1 = Layer('M1', 'drawing', connectivity=True)
M2 = Layer('M2', 'drawing', connectivity=True)
VIA = Layer('VIA1', 'drawing', connectivity=True)

print(M1, M2, VIA)

M1:drawing M2:drawing VIA1:drawing


### Building a simple cell

Add rectangles with `add_rect`. Create named references with `add_ref` and interface pins with `add_pin`.

In [15]:
# Building from outside for demonstration; in practice, use a subclass (see Section 5)
cell = LayoutCell(cell_name='SimpleCell')

# Draw two M1 rectangles
shape_a = cell.add_rect(M1, 0, 0, 1000, 500, net='A')
shape_b = cell.add_rect(M1, 2000, 0, 3000, 500, net='B')

# Named references for routing/placement
cell.add_ref('A', shape_a)
cell.add_ref('B', shape_b)

# Pins (LVS terminals) — same API, but generates GDS labels
cell.add_pin('A_PIN', shape_a)
cell.add_pin('B_PIN', shape_b)

print('Refs:', list(cell.refs.keys()))
print('Pins:', list(cell.pins.keys()))
print('Bbox:', cell.bbox(), '(nm)')

Refs: ['A', 'B', 'A_PIN', 'B_PIN']
Pins: ['A_PIN', 'B_PIN']
Bbox: (0, 0, 3000, 500) (nm)


### Compass properties

Every `Ref` has compass properties (`north`, `south`, `east`, `west`, `ne`, `nw`, `se`, `sw`, `center`)
that return `Point` objects in absolute coordinates. Points support arithmetic.

In [16]:
ref_a = cell.refs['A']

print('center:', ref_a.center)
print('north: ', ref_a.north)
print('se:    ', ref_a.se)

# Point arithmetic
offset_point = ref_a.north + (0, 200)
print('north + (0, 200):', offset_point)

center: Point(500, 250, layer=M1, net='A')
north:  Point(500, 500, layer=M1, net='A')
se:     Point(1000, 0, layer=M1, net='A')
north + (0, 200): Point(500, 700, layer=M1, net='A')


---
## 5. Hierarchy, Transforms, Placement

Layout cells form a hierarchy, just like schematic cells. Subcells are positioned using transforms
(translation, rotation, mirroring) and convenience methods `move`, `place`, and `align`.

In [17]:
class Block(LayoutCell):
    """A simple block: one M1 rectangle with two pins."""
    def __init__(self, width=2000, height=1000, **kwargs):
        super().__init__(**kwargs)
        rect = self.add_rect(M1, 0, 0, width, height, net='BODY')

        # Left and right pins
        left = self.add_rect(M1, -200, 300, 0, 700, net='L')
        right = self.add_rect(M1, width, 300, width + 200, 700, net='R')

        self.add_pin('L', left)
        self.add_pin('R', right)
        self.add_ref('BODY', rect)

### Instantiation and placement

In [18]:
class TopLayout(LayoutCell):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.B0 = Block()
        self.B1 = Block()

        # Place B1 to the right of B0 with 500nm gap
        self.B1.align('right', self.B0, margin=500)

top = TopLayout()
print('B0 bbox:', top.B0.transformed_bbox())
print('B1 bbox:', top.B1.transformed_bbox())

B0 bbox: (-200, 0, 2200, 1000)
B1 bbox: (2700, 0, 5100, 1000)


### Transforms: move, rotate, mirror

In [19]:
class TransformDemo(LayoutCell):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.ORIG = Block()

        self.MOVED = Block()
        self.MOVED.move(dx=0, dy=2000)

        self.ROTATED = Block()
        self.ROTATED.move(dx=0, dy=4000).rotate(90)

        self.MIRRORED = Block()
        self.MIRRORED.move(dx=0, dy=7000).mirror_x()

demo = TransformDemo()
for name, sub in demo.subcells.items():
    t = sub.transform
    print(f'{name:10s}  x={t.x:6d}  y={t.y:6d}  rot={t.rotation:3d}  mirror_x={t.mirror_x}')

ORIG        x=     0  y=     0  rot=  0  mirror_x=False
MOVED       x=     0  y=  2000  rot=  0  mirror_x=False
ROTATED     x=     0  y=  4000  rot= 90  mirror_x=False
MIRRORED    x=     0  y=  7000  rot=  0  mirror_x=True


### Placement with `place(at, anchor)`

`place` positions a cell so that its *anchor* ref lands exactly on the *at* target.
This is the primary way to snap cells together by their reference points.

In [20]:
class ChainDemo(LayoutCell):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.B0 = Block()
        self.B1 = Block()
        self.B2 = Block()

        # Snap B1's left pin onto B0's right pin
        self.B1.place(at=self.B0.R, anchor=self.B1.L)

        # Snap B2's left pin onto B1's right pin
        self.B2.place(at=self.B1.R, anchor=self.B2.L)

chain = ChainDemo()
for name, sub in chain.subcells.items():
    print(f'{name}: bbox = {sub.transformed_bbox()}')

B0: bbox = (-200, 0, 2200, 1000)
B1: bbox = (2000, 0, 4400, 1000)
B2: bbox = (4200, 0, 6600, 1000)


---
## 6. Routing

`route()` draws metal paths between two points. The `how` parameter controls the routing pattern:
- `'-'` — horizontal only
- `'|'` — vertical only
- `'-|'` — horizontal first, then vertical
- `'|-'` — vertical first, then horizontal

`jog_start` and `jog_end` add perpendicular offsets (in nm).

`track` and `track_end` are integer offsets that compute jog values automatically using
layer-specific spacing and width rules. For `track` to produce correct offsets, the
`LayoutCell` must implement `_get_layer_min_spacing()` (and optionally `_get_layer_min_width()`).
The base class provides a conservative fallback; PDK subclasses override these with real design rules.
See `examples/pdk/sky130/layout.py` for an example.

In [21]:
class RouteDemo(LayoutCell):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.B0 = Block()
        self.B1 = Block()
        self.B1.move(dx=5000, dy=3000)

rt = RouteDemo()

### Route patterns

In [22]:
# Horizontal-then-vertical
r1 = rt.route(rt.B0.R, rt.B1.L, layer=M1, width=200, how='-|', net='SIG')

print(f'Route: {len(r1)} segments, length = {r1.length} nm')
print(f'Start: {r1.start}')
print(f'End:   {r1.end}')

Route: 2 segments, length = 5800 nm
Start: Point(2100, 500, layer=M1, net='SIG')
End:   Point(4900, 3500, layer=M1, net='SIG')


### Route segments

A `Route` is indexable. Each segment has `.start`, `.end`, `.center` (or `.c`), `.layer`, `.net`.
Segments can be used as start/end points for subsequent routes — this is how you chain routes or
tap into the middle of an existing route.

In [23]:
for i in range(len(r1)):
    seg = r1[i]
    print(f'Segment {i}: {seg.start} → {seg.end}, center={seg.c}')

Segment 0: Point(2100, 500, layer=M1, net='SIG') → Point(4900, 500, layer=M1, net='SIG'), center=Point(3500, 500, layer=M1, net='SIG')
Segment 1: Point(4900, 500, layer=M1, net='SIG') → Point(4900, 3500, layer=M1, net='SIG'), center=Point(4900, 2000, layer=M1, net='SIG')


### Jog offsets

`jog_start` and `jog_end` shift the route perpendicular to the first/last segment.

In [24]:
class JogDemo(LayoutCell):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.B0 = Block()
        self.B1 = Block()
        self.B1.move(dx=5000, dy=3000)

        # Route with a vertical jog at the start
        self.r1 = self.route(self.B0.R, self.B1.L, layer=M1, width=200,
                             how='-|', jog_start=500, net='SIG')

jog = JogDemo()
print(f'Segments: {len(jog.r1)}  (jog adds an extra segment)')

Segments: 3  (jog adds an extra segment)


---
## 7. GDS Export & Import

A `LayerMap` maps generic layer names to GDS layer/datatype pairs.
The `GDSWriter` writes the cell hierarchy, and `GDSReader` reads it back.

In [25]:
from pade.backends.gds.layout_writer import GDSWriter
from pade.backends.gds.layout_reader import GDSReader

In [26]:
layer_map = LayerMap('tutorial', {
    'M1':   {'gds': (68, 20)},
    'M2':   {'gds': (69, 20)},
    'VIA1': {'gds': (68, 44)},
})

gds_writer = GDSWriter(layer_map=layer_map)

gds_dir = WORK_DIR / 'gds'
gds_path = gds_writer.write(rt, gds_dir)
print(f'Written: {gds_path}')

Written: ../work/gds/RouteDemo.gds


Opened in KLayout:

![Route demo in KLayout](images/route_demo_klayout.png)

### Reading back

In [27]:
reader = GDSReader(layer_map=layer_map)
imported = reader.read(gds_path)

print(f'Imported cell: {imported.cell_name}')
print(f'Subcells:      {list(imported.subcells.keys())}')
print(f'Shapes:        {len(imported.shapes)}')
print(f'Pins:          {list(imported.pins.keys())}')

Imported cell: RouteDemo
Subcells:      ['Block_0', 'Block_1']
Shapes:        2
Pins:          []


---
## 8. Schematic–Layout Integration

PADE can link layout to schematic. When a `LayoutCell` has a `schematic` reference,
it builds a **connectivity checklist** from the schematic's terminal-to-net connections.
Each `route()` call checks off the connections it covers. At the end, `check_connectivity()`
reports any missing connections.

We demonstrate this with placeholder cells — no real devices, just terminals and rectangles.

### Schematic side

In [28]:
class SubBlock(Cell):
    """A placeholder cell with two terminals."""
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.add_terminal(['A', 'B'])


class TopCircuit(Cell):
    """Top cell: two SubBlocks wired in series."""
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.add_terminal(['IN', 'OUT'])

        self.U1 = SubBlock()
        self.U2 = SubBlock()

        # Connect subcell terminals to nets
        self.U1.connect(['A', 'B'], ['IN', 'MID'])
        self.U2.connect(['A', 'B'], ['MID', 'OUT'])

schematic = TopCircuit(instance_name='TOP', cell_name='TopCircuit')

# Verify connections
for name, sub in schematic.subcells.items():
    for t in sub.get_all_terminals():
        print(f'  {name}.{t.name} → net: {t.net.name}')

  U1.A → net: IN
  U1.B → net: MID
  U2.A → net: MID
  U2.B → net: OUT


### Layout side

Each `SubBlock` gets a layout class. The top cell links to the schematic via `schematic=`.

In [29]:
class SubBlockLayout(LayoutCell):
    """Layout for SubBlock: a box with two M1 pins."""
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        body = self.add_rect(M1, 0, 0, 2000, 1000, net='BODY')
        pin_a = self.add_rect(M1, -200, 300, 0, 700, net='A')
        pin_b = self.add_rect(M1, 2000, 300, 2200, 700, net='B')

        self.add_pin('A', pin_a)
        self.add_pin('B', pin_b)
        self.add_ref('BODY', body)

In [30]:
class TopCircuitLayout(LayoutCell):
    def __init__(self, schematic, **kwargs):
        super().__init__(schematic=schematic, **kwargs)

        # Instantiate layout subcells, linked to their schematic counterparts
        self.U1 = SubBlockLayout(schematic=schematic.U1)
        self.U2 = SubBlockLayout(schematic=schematic.U2)

        # Place U2 to the right of U1
        self.U2.place(at=self.U1.B, anchor=self.U2.A)

        # Add top-level pins
        self.add_pin('IN', self.U1.A)
        self.add_pin('OUT', self.U2.B)


layout = TopCircuitLayout(schematic=schematic, cell_name='TopCircuit')

### Connectivity check — before routing

In [31]:
layout.print_connectivity_report()

Connectivity: 2/4 connections covered
  Missing net 'MID': U1.b, U2.a


The report shows missing connections. `add_pin` already checked off the terminals it covers
(U1.A via the IN pin, U2.B via the OUT pin). The internal net `MID` between U1.B and U2.A is still missing.

### Adding the missing route

In [32]:
layout.route(layout.U1.B, layout.U2.A, layer=M1, width=200, how='-', net='MID')

layout.print_connectivity_report()

Connectivity: 4/4 connections covered
  All connections covered.


All connections covered.

### Short circuit detection

`check_shorts()` flattens the layout, resolves net names through the hierarchy,
and checks for overlapping shapes with different nets on connectivity layers.

In [33]:
result = layout.check_shorts()
print(result)

3 short(s) detected:

[1] SHORT on M1: net 'MID' overlaps net 'BODY'
  shape A: TopCircuit net=MID  at (2000, 400)..(2200, 600)
  shape B: TopCircuit.U1 net=BODY  at (0, 0)..(2000, 1000)

[2] SHORT on M1: net 'BODY' overlaps net 'IN'
  shape A: TopCircuit.U1 net=BODY  at (0, 0)..(2000, 1000)
  shape B: TopCircuit.U1 net=A  at (-200, 300)..(0, 700)

[3] SHORT on M1: net 'BODY' overlaps net 'OUT'
  shape A: TopCircuit.U2 net=BODY  at (2200, 0)..(4200, 1000)
  shape B: TopCircuit.U2 net=B  at (4200, 300)..(4400, 700)


---
## 9. Summary

This tutorial covered everything in the core `pade` package:

| Area | Core classes | What they do |
|------|-------------|-------------|
| **Schematic** | `Cell`, `Testbench`, `Terminal`, `Net`, `Parameter` | Hierarchical circuit data model |
| **Stdlib** | `V`, `R`, `C`, `L`, `I`, `B` | Ideal components |
| **Simulation** | `Analysis`, `Save`, `Include`, `IC`, `Options` | Backend-agnostic statements |
| **NGspice** | `SpiceNetlistWriter`, `NgspiceSimulator`, `read_raw` | Netlist → simulate → parse results |
| **Layout** | `LayoutCell`, `Shape`, `Layer`, `Ref`, `Pin`, `Point` | Hierarchical layout with connectivity |
| **Routing** | `route()`, `Route`, `RouteSegment` | Metal path drawing |
| **GDS** | `GDSWriter`, `GDSReader`, `LayerMap` | GDSII export and import |
| **Connectivity** | `check_connectivity()`, `check_shorts()` | Schematic–layout verification |

### What core PADE does *not* provide

To go from this tutorial to a real tapeout-ready design, you need to build (or copy from `examples/`):

- **PDK layer definitions** — layer objects with GDS mappings for your target process
- **Primitive device layouts** — transistors, capacitors, resistors with correct geometry
- **Via generators** — multi-layer via stacks
- **DRC / LVS / PEX runners** — wrappers around Magic, Netgen, or commercial tools
- **Design rules** — minimum spacing, width, enclosure rules for your PDK
- **Design flow automation** — scripts to run the full pre-layout → layout → post-layout loop

Tutorial 2 (Sigma-Delta Modulator) demonstrates all of this using the SKY130 PDK.